In [184]:
import cv2
import matplotlib as plt
import numpy as np
import imutils
import math
import sklearn

In [185]:
#Reading the 2 images
A = cv2.imread("./assets/A.png", cv2.IMREAD_COLOR)
B = cv2.imread("./assets/B.png", cv2.IMREAD_COLOR)

In [186]:
#Displaying the input images
cv2.namedWindow("A", cv2.WINDOW_NORMAL)
cv2.resizeWindow("A", 700, 700)
cv2.imshow("A", A)
cv2.namedWindow("B", cv2.WINDOW_NORMAL)
cv2.resizeWindow("B", 700, 700)
cv2.imshow("B", B)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [187]:
#Function for aligning the 2 images based using the ORB Algorithm which is a feature based keypoint detection algorithm. 
def align_images(image, template, maxFeatures=100, keepPercent=0.1):
    
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    templateGray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    
    # Create the keypoint for both the images.
    orb = cv2.ORB_create(maxFeatures)
    (kpsA, descsA) = orb.detectAndCompute(imageGray, None)
    (kpsB, descsB) = orb.detectAndCompute(templateGray, None)
    
    # Matching of the features from both images 
    method = cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
    matcher = cv2.DescriptorMatcher_create(method)
    matches = matcher.match(descsA, descsB, None)
    
    # Sorting of the matches based on the distance. Shorter distance indicates more similarity. 
    matches = sorted(matches, key=lambda x:x.distance)
    
    # Keeping top 'keepPercent'% matches. This value (passed as funtion argument) was obtained experimentally for 
    #the given images. 
    keep = int(len(matches) * keepPercent)
    matches = matches[:keep]
    
    #Visualizing the matched keypoints.
    matched_pt = cv2.drawMatches(image, kpsA, template, kpsB, matches, None)
    matched_pt = imutils.resize(matched_pt, width=1000)
    cv2.imshow("Matched Keypoints", matched_pt)
    cv2.waitKey(0)
   
    #Creating the Homography Matrix for alignment of the images using Warp Perspective.
    ptsA = np.zeros((len(matches), 2), dtype="float")
    ptsB = np.zeros((len(matches), 2), dtype="float")
    

    for (i, m) in enumerate(matches):
        ptsA[i] = kpsA[m.queryIdx].pt
        ptsB[i] = kpsB[m.trainIdx].pt
   
    (H, mask) = cv2.findHomography(ptsA, ptsB, method=cv2.RANSAC)
        
    aligned = cv2.warpPerspective(image, H, template.shape[:2])

    return aligned

In [188]:
image = B.copy()
template = A.copy()

aligned = align_images(image, template, maxFeatures=500, keepPercent=0.17) 

In [189]:
aligned = imutils.resize(aligned, width=700)
template = imutils.resize(template, width=700)

In [190]:
# Visualizing the alignment by overlaying the image A and the aligned image. 
#(The aligned image is the image B.png after aligning it with A.png)
overlay = template.copy()
output = aligned.copy()
cv2.addWeighted(overlay, 0.5, output, 0.5, 0, output)

cv2.imshow("Overlay of the Aligned Images", output)

#For comparison, the original images overlayed with each other are also visualised.
X = A.copy()
Y = B.copy()
cv2.addWeighted(Y, 0.5, X, 0.5, 0, Y)

cv2.namedWindow("Overlay of the Original Images", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Overlay of the Original Images", 700, 700)
cv2.imshow("Overlay of the Original Images", Y)
cv2.waitKey(0)


-1

In [191]:
#Function for coorecting for the environmental factors. 
#The Dark Channel Prior (DCP) algorithm is used to correct the haze and fog. 

def env_correction(img_n, img, kernel_size = 5, omega = 10, tx = 0.5 ):        # 'img_n' represents normalized form 
                                                                       # of the original image 'img'
    
    # Finding the Dark Chaneel with a 10x10 kernel. 
    b,g,r = cv2.split(img_n)
    dc = cv2.min(cv2.min(r,g),b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(kernel_size,kernel_size))
    dark = cv2.erode(dc,kernel)
    
    [h,w] = img_n.shape[:2]
    img_size = h*w
    numpx = int(max(math.floor(img_size/1000),1))
    darkvec = dark.reshape(img_size)
    imvec = img_n.reshape(img_size,3)

    # Sorting the dark channel pixels
    indices = darkvec.argsort()
    indices = indices[img_size-numpx::]
    
    # Claculating the atmospheric light
    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
        atmsum = atmsum + imvec[indices[ind]]
    A_light = atmsum / numpx
    
    # Normalizing the image with the atmospheric light.
    im3 = np.empty(img_n.shape,img_n.dtype)
    for ind in range(0,3):
        im3[:,:,ind] = img_n[:,:,ind]/A_light[0,ind]

    transmission = 1 - omega*dark      # Estimation of the transmission using the dark channel.
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray)/255
    r = 100
    eps = 0.1
    
    # Computing the guided filter for transmission estimation
    mean_I = cv2.boxFilter(gray,cv2.CV_64F,(r,r))
    mean_p = cv2.boxFilter(transmission, cv2.CV_64F,(r,r))
    mean_Ip = cv2.boxFilter(gray*transmission,cv2.CV_64F,(r,r))
    cov_Ip = mean_Ip - mean_I*mean_p

    mean_II = cv2.boxFilter(gray*gray,cv2.CV_64F,(r,r))
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    t = mean_a*gray + mean_b;
    
    res = np.empty(img_n.shape,img_n.dtype);
    t = cv2.max(t,tx);                      # Clipping the transmission to the min. value
    
    # Finding the corrected image after application of transmission and atmospheric light.
    for ind in range(0,3):
        res[:,:,ind] = (img_n[:,:,ind]-A_light[0,ind])/t + A_light[0,ind]
        
    return res

In [192]:
# Normalization of the image. 
A_n = A.astype('float64')/255;
aligned_n = aligned.astype('float64')/255;

A_corrected = env_correction(A_n, A, kernel_size = 10, omega = 10, tx = 0.6)
A_corrected = cv2.resize(A_corrected, (700, 700))
aligned_corrected = env_correction(aligned_n, aligned, kernel_size = 8, omega = 10, tx = 0.6)

# A_corrected = A.astype('float32');
# aligned_corrected = A.astype('float32');


In [193]:
# Visualization of the original and corrected version of A.png

cv2.namedWindow("A", cv2.WINDOW_NORMAL)
cv2.resizeWindow("A", 700, 700)
cv2.imshow("A", A)
cv2.namedWindow("A_corrected", cv2.WINDOW_NORMAL)
cv2.resizeWindow("A_corrected", 700, 700)
cv2.imshow("A_corrected", A_corrected)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [194]:
# Visualization of the original and corrected version of the aligned B.png

cv2.namedWindow("aligned", cv2.WINDOW_NORMAL)
cv2.resizeWindow("aligned", 700, 700)
cv2.imshow("aligned", aligned)
cv2.namedWindow("aligned_corrected", cv2.WINDOW_NORMAL)
cv2.resizeWindow("aligned_corrected", 700, 700)
cv2.imshow("aligned_corrected", aligned_corrected)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [195]:
# Visualization of the both the corrected images.

cv2.namedWindow("A_corrected", cv2.WINDOW_NORMAL)
cv2.resizeWindow("A_corrected", 700, 700)
cv2.imshow("A_corrected", A_corrected)
cv2.namedWindow("aligned_corrected", cv2.WINDOW_NORMAL)
cv2.resizeWindow("aligned_corrected", 700, 700)
cv2.imshow("aligned_corrected", aligned_corrected)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [196]:
# Converting the corrected images to gray scale for further processing for finding the differences. 
gray_A = cv2.cvtColor(A_corrected.astype('float32'), cv2.COLOR_BGR2GRAY)
gray_aligned = cv2.cvtColor(aligned_corrected.astype('float32'), cv2.COLOR_BGR2GRAY)

In [197]:
# Visualization of the gray scale versions of the images

cv2.namedWindow("gray_A", cv2.WINDOW_NORMAL)
cv2.resizeWindow("gray_A", 700, 700)
cv2.imshow("gray_A", gray_A)
cv2.namedWindow("gray_aligned", cv2.WINDOW_NORMAL)
cv2.resizeWindow("gray_aligned", 700, 700)
cv2.imshow("gray_aligned", gray_aligned)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [198]:
#Finding the differences using a series of morphological operations. 

kernel = np.ones((11, 11), np.uint8)
A_dilation = cv2.dilate(gray_A, kernel, iterations=1)
aligned_dilation = cv2.dilate(gray_aligned, kernel, iterations=1)

# A_erode = cv2.erode(gray_A, kernel, iterations=1)
# aligned_erode = cv2.erode(gray_aligned, kernel, iterations=1)

In [199]:
cv2.namedWindow("aligned_dilation", cv2.WINDOW_NORMAL)
cv2.resizeWindow("aligned_dilation", 700, 700)
cv2.imshow("aligned_dilation", aligned_dilation)
cv2.namedWindow("A_dilation", cv2.WINDOW_NORMAL)
cv2.resizeWindow("A_dilation", 700, 700)
cv2.imshow("A_dilation", A_dilation)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [200]:
diff = cv2.subtract(aligned_dilation, A_dilation)

#Displaying the crude difference between the Images.
cv2.imshow("Difference Between the 2 Images", diff)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [201]:
diff = (diff * 255).astype("uint8")

In [202]:
# Thresholding the difference image and finding the contours to mark the differences. 

ret, thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_TOZERO_INV + cv2.THRESH_OTSU)
# thresh = cv2.medianBlur(thresh,5)
kernel = np.ones((11, 11), np.uint8)
thresh = cv2.erode(thresh, kernel, iterations=1)      #The erosion and dilation together performing opening of the image.
thresh = cv2.dilate(thresh, kernel, iterations=1)

cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

In [203]:
cv2.imshow("Threshold", thresh)
cv2.waitKey(0)

-1

In [204]:
A_marked = A_corrected.copy()
B_marked = aligned_corrected.copy()

# Marking the differences with boxes from the obtained contours.
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(A_marked, (x, y), (x + w, y + h), (0, 255, 255), 1)
    cv2.rectangle(B_marked, (x, y), (x + w, y + h), (0, 255, 255), 2)

stacked = np.hstack([A_marked, B_marked])
cv2.imshow("Marked differences between A and B", stacked)
cv2.waitKey(0)

-1

In [205]:
pic_n_A = A_corrected.copy()
pic_n_B = aligned_corrected.copy()


# reshaping the images into 2D array for application of the k-means clustering algo.  
pic_n_A = pic_n_A.reshape(A_corrected.shape[0]*A_corrected.shape[1], A_corrected.shape[2])
pic_n_B = pic_n_B.reshape(aligned_corrected.shape[0]*aligned_corrected.shape[1], aligned_corrected.shape[2])


In [206]:
from sklearn.cluster import KMeans

In [207]:
# Applying k-means classifier to label or mark the pixels into k no. of classes. (Here, k = 3)
kmeans_A = sklearn.cluster.KMeans(n_clusters=3, n_init="auto", random_state=0).fit(pic_n_A)
pic_n_A = kmeans_A.cluster_centers_[kmeans_A.labels_]
kmeans_B = sklearn.cluster.KMeans(n_clusters=3, n_init="auto", random_state=0).fit(pic_n_B)
pic_n_B = kmeans_B.cluster_centers_[kmeans_B.labels_]

In [208]:
# Again reshaping the clustered images. 
cluster_pic_A = pic_n_A.reshape(A_corrected.shape[0], A_corrected.shape[1], A_corrected.shape[2])
cluster_pic_B = pic_n_B.reshape(aligned_corrected.shape[0], aligned_corrected.shape[1], aligned_corrected.shape[2])

In [209]:
# Visualization of the images after k-means clustering.
cv2.imshow("cluster_pic_A", cluster_pic_A)
cv2.imshow("cluster_pic_B", cluster_pic_B)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [210]:
A_corrected.shape

(700, 700, 3)

In [211]:
# Segmenting the image based on the labels assigned to each pixel by the k-means clustering algorithm. 
cluster_pic_Ac = aligned_corrected.copy()
i=0
for x in range(0,700):
        for y in range(0,700):
            for c in range(1):
                if (kmeans_A.labels_[i] == 0 ):                     
                    cluster_pic_Ac[x,y] = [0,0,255]             # b g r
                elif (kmeans_A.labels_[i] == 1 ):                     
                    cluster_pic_Ac[x,y] = [90,0,100]             # b g r
                elif (kmeans_A.labels_[i] == 2 ):                     
                    cluster_pic_Ac[x,y] = [0,255,0]           # b g r
                else:
                    cluster_pic_Ac[x,y] = [0,0,0]
                    pass
                i = i+1
cv2.imshow("cluster_pic_Ac", cluster_pic_Ac)
cv2.waitKey(0)

-1

In [212]:
# Thresholding to highlight the Segmented changes. 
z = cluster_pic_Ac.copy()
for x in range(0,700):
        for y in range(0,700):
            for c in range(1):
                if (thresh[x,y]*255 < 100 ):                    
                    z[x,y] = [0,0,0]

In [213]:
# Adding classification legends to the output image and marking the contours around them for better visualization.
fontScale = 0.5
thickness = 2
color = (255,255,255)

for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(z, (x, y), (x + w, y + h), (255, 255, 255), 2)

In [214]:
# Displaying the marked differences and the segmented classifications. 
cv2.imshow("Marked Differences", stacked)
cv2.imshow("Segmented Differences", z)
cv2.waitKey(0)

-1